In [1]:
from spacecraft.satellite import SatelliteImplementation
from setup.initial_settings import SimulationSetupReader
from setup.two_line_element import TwoLineElementReader
from visualziations.visualizations import plot_orbit, plot_lla, plot_position, plot_magnetic_field_eci, plot_magnetic_field_sbf, plot_sun_vector_eci, plot_sun_vector_sbf
from visualziations.visualizations import plot_angular_velocity, plot_euler_angles, plot_pointing_error, plot_torque, plot_angular_acceleration
from spacecraft.sensors import MagnetometerImplementation, SunsensorImplementation, SensorFusionImplementation
import core.transformations as tr
import core.utilities as ut
from pathlib import Path

import numpy as np

setup = SimulationSetupReader(Path('setup/initial_settings_tmp.json'))
tle = TwoLineElementReader(Path('setup/tle'))
magnetometer = MagnetometerImplementation(noise=True)
sunsensor = SunsensorImplementation(noise=True)
sensor_fusion = SensorFusionImplementation(['triad', 'quest', 'ekf'], tr.euler_xyz_to_quaternion(setup.euler_angles))

satellite = SatelliteImplementation(setup, tle, magnetometer, sunsensor, sensor_fusion)
quaternion_prev = satellite.quaternion.copy()
satellite.state_vector.next_row()
ut.basic_state_vector(satellite)

n_iter = 6000
for x in range(1, n_iter, 1):
    satellite.update_iteration(x)
    satellite.apply_rotation()

    mag_field_sbf, mag_field_eci = satellite.magnetic_field
    sun_vector_sbf, sun_vector_eci = satellite.sun_vector

    if x % 100 == 0:
        print(f"Iteration {x} of {n_iter}")
        # print(f"Magnetic field (SBF): {mag_field_sbf}")
        # print(f"Magnetic field (ECI): {mag_field_eci}")
    # print(f"Reference quaternion: {q_ref}"

    # sa tellite.apply_triad(
    #     ut.normalize(mag_field_eci), ut.normalize(sun_vector_eci), 
    #     ut.normalize(mag_field_sbf), ut.normalize(sun_vector_sbf)
    # )
    # q_triad = satellite.quaternion.copy()
    # print(f"Quaternion after TRIAD: {q_triad}")
    # print(f"Rotation {np.arccos(q_triad[3]) * 2 * 180 / np.pi}")

    # satellite.apply_quest(
    #     [ut.normalize(mag_field_sbf), ut.normalize(sun_vector_sbf)],
    #     [ut.normalize(mag_field_eci), ut.normalize(sun_vector_eci)]
    # )
    # q_quest = satellite.quaternion.copy()
    # print(f"Quaternion after QUEST: {q_quest}")
  
    # ekf automatically updates the quaternion by the obtained rotation
    satellite.apply_ekf(
        [ut.normalize(mag_field_sbf), ut.normalize(sun_vector_sbf)],
        [ut.normalize(mag_field_eci), ut.normalize(sun_vector_eci)],
        quaternion_prev
    )
    # q_ekf = satellite.quaternion 

    satellite.apply_detumbling(
        adapt_velocity=False, 
        adapt_magnetic=False, 
        proportional=False, 
        modified=False
    )
    satellite.manage_modes()
    satellite.apply_pointing(
        'earth_pointing', 
        align_axis=[0, 0, 1]
    )

    # print(f"Quaternion after EKF: {q_ekf}")
    satellite.state_vector.next_row()
    ut.basic_state_vector(satellite)
    quaternion_prev = satellite.quaternion.copy() 

satellite.state_vector.to_csv('simulation_state_vector.csv')

print(satellite.state_vector.to_dataframe()) 
plot_orbit(satellite.state_vector.to_dataframe(), setup)
plot_position(satellite.state_vector.to_dataframe())
plot_lla(satellite.state_vector.to_dataframe())
plot_magnetic_field_eci(satellite.state_vector.to_dataframe())
plot_magnetic_field_sbf(satellite.state_vector.to_dataframe()) 
plot_angular_velocity(satellite.state_vector.to_dataframe())
plot_euler_angles(satellite.state_vector.to_dataframe())
plot_pointing_error(satellite.state_vector.to_dataframe())
plot_torque(satellite.state_vector.to_dataframe())
plot_angular_acceleration(satellite.state_vector.to_dataframe())
plot_sun_vector_eci(satellite.state_vector.to_dataframe())
plot_sun_vector_sbf(satellite.state_vector.to_dataframe())


Detumbling stopped (|ω|=0.00 deg/s). Pointing started.
Iteration 100 of 6000
Iteration 200 of 6000
Iteration 300 of 6000
Iteration 400 of 6000
Iteration 500 of 6000
Iteration 600 of 6000
Iteration 700 of 6000
Iteration 800 of 6000
Iteration 900 of 6000
Iteration 1000 of 6000
Iteration 1100 of 6000
Iteration 1200 of 6000
Iteration 1300 of 6000
Iteration 1400 of 6000
Iteration 1500 of 6000
Iteration 1600 of 6000
Iteration 1700 of 6000
Iteration 1800 of 6000
Iteration 1900 of 6000
Iteration 2000 of 6000
Iteration 2100 of 6000
Iteration 2200 of 6000
Iteration 2300 of 6000
Iteration 2400 of 6000
Iteration 2500 of 6000
Iteration 2600 of 6000
Iteration 2700 of 6000
Iteration 2800 of 6000
Iteration 2900 of 6000
Iteration 3000 of 6000
Iteration 3100 of 6000
Iteration 3200 of 6000
Iteration 3300 of 6000
Iteration 3400 of 6000
Iteration 3500 of 6000
Iteration 3600 of 6000
Iteration 3700 of 6000
Iteration 3800 of 6000
Iteration 3900 of 6000
Iteration 4000 of 6000
Iteration 4100 of 6000
Iteration 4

In [1]:
# TODO
# complete b-cross
# add bang-bang mode for b-dot
# test all b-dot modes
# move all initial parameters to settings (sensors, actuators, satellite, simulation)
# add on-off time for actuators and sensors 
# use better visualizations (interactive, zoom in/out   )
# some optimization
# verify the descriptions and documentation

# OPTIONAL
# add more sensors
# add more actuators


Documentation:

1. Whats in this code

2. Short intro
    what is adcs
    simulating environment
    simulating sensors and actuartors
    basic control

3. Threoretical background
    simlating envireonment (tle, magnetic field, sun position etc)
    reference frames and transformations
    sensors (magnetometer, sun sensor, star tracker, gyroscope)


SyntaxError: invalid syntax (1693032554.py, line 16)

In [7]:
import numpy as np
#                      x                 y                   z              x               y
array1 = np.array([-0.0114, -        -0.0114,          0.0114]) #         -0.0114, -        0.0114
array2 = np.array([ 1.92709051e-05, -3.48258106e-05,  1.31687787e-05]) # 1.92709051e-05, -3.48258106e-05

array3 = [
    (array1[1]*array2[2] - array1[2]*array2[1]),
    (array1[2]*array2[0] - array1[0]*array2[2]),
    (array1[0]*array2[1] - array1[1]*array2[0])
]

array3_np = np.cross(array1, array2)    

print(array3)
print(array3_np)


[np.float64(5.4713831802e-07), np.float64(3.6981239532e-07), np.float64(1.7732592269999998e-07)]
[5.47138318e-07 3.69812395e-07 1.77325923e-07]
